### ResNet34: Face recognition using a binary classifier
The framework only implements `compare_faces`, which tells me if two faces are the same person.  

The naive approach, that I am investigating here, is to compare the test_img to every img in the training dataset.  

Since multiple images can be a match, the classifcation tolerance is lowered (using a binary search) until exactly one result is given. This is then called the prediction of ResNet34. 

In [53]:
from src.DataLoader.load_lfw import load_lfw

In [54]:
training_filepaths, training_labels, test_filepaths, test_labels = load_lfw()

In [121]:
class TestImageNoFaceException(Exception):
    """If there is no face found on the test image"""

def predict_all_resnet(test_filepath, training_filepaths, training_labels, tolerance = 0.6):
    """
    Compares the test image to all training images. 
    Returns a list of matches according to the given tolerance.
    """
    test_img = load_image_file(test_filepath)
    test_ecodings = np.array(face_encodings(test_img))
    if test_ecodings.shape[0] == 0:
            raise TestImageNoFaceException()
    matching_filepaths = []
    predictions = []
    
    for i in range(len(training_filepaths)):
        train_img = load_image_file(training_filepaths[i])
        train_ecodings = np.array(face_encodings(train_img))
        if train_ecodings.shape[0] == 0:
            # TODO: Image didnt contain a face
            continue

        if compare_faces(test_ecodings, train_ecodings, tolerance = tolerance)[0]:
                matching_filepaths.append(training_filepaths[i])
                predictions.append(training_labels[i])
    return matching_filepaths, predictions


def predict_resnet(test_filepath, training_filepaths, training_labels):
    """
    Performs binary search using a binary classifier. 
    Returns the single most predicted image.
    """
    upper_bound = 1
    lower_bound = 0
    tolerance = 0.30
    for i in range(10):
        
        try:
            matching_filepaths, predictions = predict_all_resnet(test_filepath, training_filepaths, training_labels, tolerance)
        except TestImageNoFaceException:
            return [[None],[-1]]
        print(f'tolerance: {tolerance}')
        print(f'predictions: {predictions}')
        if len(predictions) > 1:
            upper_bound = tolerance
            tolerance = (tolerance - lower_bound)*0.5
        if len(predictions) < 1:
            lower_bound = tolerance
            tolerance *= upper_bound + (upper_bound - tolerance)*0.5
        if len(predictions) == 1:
            return matching_filepaths, predictions
        
        if len(matching_filepaths) > 0:
            training_filepaths = matching_filepaths
            training_labels = predictions
        
    return matching_filepaths, predictions

i = 99
print(f'actual class: {test_labels[i]}')
predict_resnet(test_filepaths[i], training_filepaths, training_labels)

actual class: 5281
tolerance: 0.3
predictions: []
tolerance: 0.405
predictions: [5281]


(['dat/lfw/Tony_Blair/extracted_faces/Tony_Blair_0138_0.png'], [5281])

### Computing Recognition Accuracy

In [122]:
predicted_filepaths = []
predicted_labels = []
for i in range(len(test_filepaths)):
    if i % 10 == 0:
        print(i)
    pred_filepaths, pred_labels = predict_resnet(test_filepaths[i], training_filepaths, training_labels)
    predicted_filepaths.extend(pred_filepaths)
    predicted_labels.extend(pred_labels)
    predictions.append(res)

0
tolerance: 0.3
predictions: [4807]
tolerance: 0.3
predictions: [4517]
tolerance: 0.3
predictions: [776]
tolerance: 0.3
predictions: [4805]
tolerance: 0.3
predictions: [973]
tolerance: 0.3
predictions: [2492]
tolerance: 0.3
predictions: [3055]
tolerance: 0.3
predictions: [1780]
tolerance: 0.3
predictions: [1809]
tolerance: 0.3
predictions: [2727]
10
tolerance: 0.3
predictions: [1354]
tolerance: 0.3
predictions: [1809]
tolerance: 0.3
predictions: [1809]
tolerance: 0.3
predictions: [473]
tolerance: 0.3
predictions: [4447]
tolerance: 0.3
predictions: [4185]
tolerance: 0.3
predictions: [5281]
tolerance: 0.3
predictions: [4024]
tolerance: 0.3
predictions: [1949]
tolerance: 0.3
predictions: [1809]
20
tolerance: 0.3
predictions: [4143]
tolerance: 0.3
predictions: [1809]
tolerance: 0.3
predictions: [3251]
tolerance: 0.3
predictions: [2393]
tolerance: 0.3
predictions: [3518]
tolerance: 0.3
predictions: [980]
tolerance: 0.3
predictions: [3914, 3914]
tolerance: 0.15
predictions: [3914]
tolerance

In [123]:
from sklearn.metrics import accuracy_score
accuracy_score(predicted_labels, test_labels)

ValueError: Found input variables with inconsistent numbers of samples: [89, 100]

In [124]:
predicted_labels

[4807,
 4517,
 776,
 4805,
 973,
 2492,
 3055,
 1780,
 1809,
 2727,
 1354,
 1809,
 1809,
 473,
 4447,
 4185,
 5281,
 4024,
 1949,
 1809,
 4143,
 1809,
 3251,
 2393,
 3518,
 -1,
 980,
 3914,
 2658,
 5252,
 5355,
 926,
 4761,
 2167,
 362,
 4365,
 4860,
 4037,
 1651,
 4223,
 2491,
 1593,
 2105,
 4571,
 4473,
 3641,
 2580,
 1522,
 2067,
 2659,
 -1,
 2853,
 1011,
 3109,
 2715,
 1027,
 681,
 1944,
 4795,
 413,
 5271,
 4411,
 4847,
 1998,
 1011,
 1512,
 2805,
 2530,
 3802,
 3914,
 4807,
 1354,
 5281,
 2455,
 295,
 1809,
 1809,
 1830,
 -1,
 1830,
 1809,
 1354,
 5281,
 3641,
 1354,
 4905,
 1804,
 1809,
 5281]